In [9]:
# -*- encoding: utf-8 -*-
'''
@File    :   address_API.py
@Time    :   2020/09/07 15:57:42
@Author  :   DataMagician 
@Version :   1.0
@Contact :   408903228@qq.com
'''

# here put the import lib
import requests
import json
import pandas as pd
import numpy as np

def getUrl(*address):
    '''
    调用地图API获取待查询地址专属url
    最高查询次数30w/天，最大并发量160/秒
    '''
    ak = 'Vhzgfjih2w3YY0WUPuPdKiMQKwIor9rp'
    if len(address) < 1:
        return None
    else:
        for add in address:   
            url = 'http://api.map.baidu.com/geocoding/v3/?address={inputAddress}&output=json&ak={myAk}'.format(inputAddress=add,myAk=ak)  
            yield url
            
def getPosition(url):
    '''返回经纬度信息'''
    res = requests.get(url)
    json_data = json.loads(res.text)
    
    if json_data['status'] == 0:
        lat = json_data['result']['location']['lat'] #纬度
        lng = json_data['result']['location']['lng'] #经度
        precise = json_data['result']['precise']
        confidence = json_data['result']['confidence']
        comprehension = json_data['result']['comprehension']
        level = json_data['result']['level']
    else:
        print("Error output!")
        return json_data['status']
    return lat,lng,precise,confidence,comprehension,level

def getStrings(address):
    for add in address:
        add_url = list(getUrl(add))[0]
        print(add_url)
        try:
            lat,lng,precise,confidence,comprehension,level = getPosition(add_url) 
            print("地址：{0}|经度:{1}|纬度:{2}|精度:{3}|置信度:{4}|解释性:{5}|分类等级:{6}".format(add,lat,lng,precise,confidence,comprehension,level))
        except Error as e:
            print(e)

def getDict(address):
    column = ["地址","经度","纬度","精度","置信度","解释性","分类等级"]
    addressL,latL,lngL,preciseL,confidenceL,comprehensionL,levelL = [],[],[],[],[],[],[]
    n = -1
    for add in address:
        addressL.append(add)
        lat,lng,precise,confidence,comprehension,level  = getPosition(list(getUrl(add))[0])
        latL.append(lat)
        lngL.append(lng)
        preciseL.append(precise)
        confidenceL.append(confidence)
        comprehensionL.append(comprehension)
        levelL.append(level)
                
    return {cn : data for cn , data in zip(column,(addressL,latL,lngL,preciseL,confidenceL,comprehensionL,levelL))}


# 输入目的地

In [10]:
str_ = "杭州火车站-杭州灵隐寺-浙江省博物馆-浙江省图书馆-杭州晓书馆-西湖区蚂蚁金服-浙江省金华市东阳市横店镇-西湖景区-西湖青之坞-杭州小河直街"

# 爬取坐标地址

In [11]:
location_info = getDict(str_.split("-"))
df = pd.DataFrame(location_info)
df 

,地址,经度,纬度,精度,置信度,解释性,分类等级
0,杭州火车站,30.249207,120.189606,0,50,100,火车站
1,杭州灵隐寺,30.246569,120.108260,0,50,100,旅游景点
2,浙江省博物馆,30.257342,120.149943,0,50,0,旅游景点
3,浙江省图书馆,39.950781,116.328379,0,70,0,教育
4,杭州晓书馆,30.371017,120.039324,1,70,100,教育
5,西湖区蚂蚁金服,30.279204,120.131800,1,80,100,公司企业
6,浙江省金华市东阳市横店镇,29.161567,120.350404,0,25,100,乡镇
7,西湖景区,30.228932,120.127920,0,50,100,旅游景点
8,西湖青之坞,30.278984,120.153887,0,50,100,NoClass
9,杭州小河直街,30.313845,120.141920,0,30,100,道路


# 排序所有点

In [12]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)

def minkowski_distance(vec1, vec2, p=3):
    """
    闵氏距离
    当p=1时，就是曼哈顿距离
    当p=2时，就是欧氏距离
    当p→∞时，就是切比雪夫距离
    :param vec1:
    :param vec2:
    :param p:
    :return:
    """
    # return sum([(x - y) ** p for (x, y) in zip(vec1, vec2)]) ** (1 / p)
    return np.linalg.norm(vec1 - vec2, ord=p)

def sorted_by_distance(df=df,my_p=1):
    temp = np.array([[i,j,z] for i,j,z in zip(df.纬度.to_list(),df.经度.to_list(),df.地址)])
    dict_ = [[a[-1],b[-1],minkowski_distance(a[:-1].astype(float),b[:-1].astype(float),p=my_p)] for b in temp for a in temp]
    distance_name = "distance"
    table = pd.DataFrame(dict_,columns=["start","end",distance_name ])
    newtable1 = table.sort_values(by=distance_name , ascending=True)
    newtable = newtable1[newtable1["start"] != newtable1["end"]]
    return newtable.reset_index(drop=True)

print("曼哈顿距离排序")
sorted_data = sorted_by_distance(my_p=1)
sorted_data

曼哈顿距离排序


,start,end,distance
0,西湖区蚂蚁金服,西湖青之坞,0.022307
1,西湖青之坞,西湖区蚂蚁金服,0.022307
2,浙江省博物馆,西湖青之坞,0.025586
3,西湖青之坞,浙江省博物馆,0.025586
4,杭州灵隐寺,西湖景区,0.037297
5,西湖景区,杭州灵隐寺,0.037297
6,浙江省博物馆,西湖区蚂蚁金服,0.040005
7,西湖区蚂蚁金服,浙江省博物馆,0.040005
8,杭州小河直街,西湖区蚂蚁金服,0.044762
9,西湖区蚂蚁金服,杭州小河直街,0.044762


# 排序所有点的集合，确定初始落脚位置

In [13]:
distance_tb = sorted_by_distance(my_p=1)
distance_from_any_where = {address:(distance_tb[distance_tb.start==address].distance).sum() for address in set(distance_tb.start)}
distance_df = pd.merge(df,pd.DataFrame(sorted(distance_from_any_where.items(),key= lambda x : x[-1]),columns=["地址","距离所有点的距离分"]),how='right', left_on=u'地址', right_on='地址')
distance_df.columns = ['地址',  '纬度', '经度','精度', '置信度', '解释性', '分类等级', '距离所有点的距离分']
distance_df

,地址,纬度,经度,精度,置信度,解释性,分类等级,距离所有点的距离分
0,西湖区蚂蚁金服,30.279204,120.131800,1,80,100,公司企业,15.300732
1,浙江省博物馆,30.257342,120.149943,0,50,0,旅游景点,15.316337
2,西湖青之坞,30.278984,120.153887,0,50,100,NoClass,15.332112
3,杭州灵隐寺,30.246569,120.108260,0,50,100,旅游景点,15.413514
4,杭州小河直街,30.313845,120.141920,0,30,100,道路,15.439297
5,西湖景区,30.228932,120.127920,0,50,100,旅游景点,15.440695
6,杭州火车站,30.249207,120.189606,0,50,100,火车站,15.562699
7,杭州晓书馆,30.371017,120.039324,1,70,100,教育,16.282336
8,浙江省金华市东阳市横店镇,29.161567,120.350404,0,25,100,乡镇,25.504381
9,浙江省图书馆,39.950781,116.328379,0,70,0,教育,122.608014


# 可视化

In [ ]:
def sigmod(z):
    return 1/(1+np.exp(-z))

import plotly as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from IPython.display import Image 
import cufflinks as cf
cf.go_offline()###这两句是离线生成图片的设置
cf.set_config_file(offline=True, world_readable=True)
init_notebook_mode(connected=True)
%matplotlib inline
def plot_Map_IceFire(df=df,col="距离所有点的距离分",size=10,map_name="曼哈顿距离相对最优路径规划"):
    token='pk.eyJ1Ijoic3l2aW5jZSIsImEiOiJjazZrNTcwY3kwMHBrM2txaGJqZWEzNWExIn0.tLQHY_OoiR2NMxnYHXUBAA'
    #token = 'pk.eyJ1IjoiYmxhY2tzaGVlcHdhbGwwMzA1IiwiYSI6ImNrMHo5ZnQxYjBjbG8zbm84b3hrb25vb24ifQ.K8tcDjJDsPcjdYFTSVgTxw'
    #token = 'pk.eyJ1IjoibWFubWFuemhhbmciLCJhIjoiY2thMDRkbGM3MDh0aDNybjZpM3hyam5yOCJ9.TOaUXDGA4hzIoAowDRSOqw'
    px.set_mapbox_access_token(token)
    fig = px.scatter_mapbox(df
                        , lat=df["纬度"]
                        , lon=df["经度"]
                        , color=df["距离所有点的距离分"]#np.exp(df["距离所有点的距离分"]-df["距离所有点的距离分"].mean())
                        , size=df["距离所有点的距离分"]
                        , hover_name='地址' # 点的名称
                        , hover_data=['置信度','距离所有点的距离分','分类等级'] # 显示的内容
                        , color_continuous_scale=px.colors.cyclical.IceFire
                        , size_max=16
                        , zoom=10
                        , center = {'lon': df["经度"].mean(), 'lat': df['纬度'].mean()}
                        )
    
    # 画链接线
    fig.add_trace(go.Scattermapbox(mode='lines',#标记加链接模式 
                               lon=df["经度"],
                               lat=df['纬度'],
                               marker={'size':3,'color':'#43CD80' },#标记的大小和颜色信息 
                               hovertext=df["分类等级"], #标记的文字说明
                               hoverinfo='text',#标记的文字类型
                               showlegend=False))
                               

    fig.update_layout(mapbox={'accesstoken':token,'center':{'lon': np.argmax(np.bincount(df["经度"])), 'lat':np.argmax(np.bincount(df["纬度"]))},'zoom':10.3,'style':'outdoors'},title={'text':map_name,'xref':'paper','x':0.5,},margin={'l':10,'r':0,'t':50,'b':10})
    fig.show()
    
plot_Map_IceFire(distance_df)